In [31]:
import requests
import cchardet
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime, timedelta, date
from copy import deepcopy
from tqdm.notebook import tqdm
import pickle
import sys
import os
import re
from transliterate import translit
import json
import traceback
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
import shutil

In [2]:
url = 'https://allgosts.ru'
NEW_DOWNLOAD_FOLDER = 'C:/Users/1645286/vkr/files'

In [3]:
global_dict = {}

In [4]:
res = requests.get(url)
main = bs(res.text, 'lxml')
main_list = main.find('ul', class_='content-list').findAll('li', class_='content-list_item')
main_list = [(li.find('a')['href'], li.find('a').text) for li in main_list]

In [5]:
global_dict = {
    el[1]: dict() for el in main_list
}

In [6]:
status_dict = {
    'Действует': 'active',
    'Заменен': 'replaced',
    'Отменен': 'cancelled'
}

In [7]:
N_MAX_DOC = 50
main_start = 0
section_start = 0
doc_start = 0

In [8]:
def move_file(source_path, destination_path):
    file_name = os.path.basename(source_path)
    new_path = os.path.join(destination_path, file_name)
    
    if os.path.exists(new_path):
        base_name, extension = os.path.splitext(file_name)
        counter = 1
        while os.path.exists(new_path):
            new_file_name = f"{base_name} ({counter}){extension}"
            new_path = os.path.join(destination_path, new_file_name)
            counter += 1
    
    os.replace(source_path, new_path)
    return os.path.basename(new_path)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--blink-settings=imagesEnabled=false")
chrome_options.add_argument("--disable-crash-reporter")
chrome_options.add_argument("--disable-low-res-tiling")
chrome_options.add_argument("--disable-renderer-backgrounding")
chrome_options.add_argument("--disable-oopr-debug-crash-dump")
chrome_options.add_argument("--no-crash-upload")
driver = uc.Chrome(headless=False, use_subprocess=False, user_multi_procs=False, options=chrome_options)

not_downloaded = list()
max_wait_time = 10
sleep_time = 1

for main_href, main_name in tqdm(main_list[main_start:], desc='Global list'):
    section = requests.get(url + main_href)
    section = [
        (li.find('a')['href'], li.find('a').text)
        for li in bs(section.text, 'lxml').find('ul', class_='content-list').findAll('li', class_='content-list_item')
    ]
    for section_href, section_name in tqdm(section[section_start:], leave=False, desc='Sections'):
        global_dict[main_name][section_name] = list()
        section_docs = requests.get(url + section_href)
        if section_docs.status_code == 404:
            continue

        lis = bs(section_docs.text, 'lxml').find('ul', class_='content-list').findAll('li', class_='content-list_item')
        if lis:
            section_docs = [(li.find('a')['href'], li.find('a').text) for li in lis]
        else:
            section_docs = []
        for doc_href, doc_title in tqdm(section_docs[doc_start:N_MAX_DOC], leave=False, desc='Section documents'):
            doc_link = url + doc_href
            doc = requests.get(doc_link)
            if doc.status_code == 404:
                continue

            doc = bs(doc.text, 'lxml')
            upper_doc_section = doc.find('div', class_='section')
            rows_raw = upper_doc_section.findAll('div', class_='row')
            rows = {
                row.find('div', class_='label').text.strip()[:-1]: row.find('div', class_='value').text.strip()
                for row in rows_raw
            }
            link_row = [row for row in rows_raw if 'Скачать:' in row.find('div', class_='label').text]
            filename = ''
            if link_row:
                file_type = link_row[0].find('a', {'id': 'pdf_download'})['id'] or link_row[0].find('a', {'id': 'doc_download'})['id']
                download_name = link_row[0].find('a', {'id': 'pdf_download'})['href'] or link_row[0].find('a', {'id': 'doc_download'})['href']
                if file_type:
                    download_name = download_name.split('/')[-1]
                    driver.get(doc_link)
                    WebDriverWait(driver, 5) \
                        .until(EC.presence_of_element_located((By.ID, file_type)))
                    driver.find_element(By.ID, file_type).click()
                    download_path = f'C:/Users/1645286/Downloads/{download_name}'
                    c = 1
                    max_retries = max_wait_time / sleep_time
                    while not os.path.exists(download_path) and c < max_retries:
                        time.sleep(sleep_time)
                        c += 1
                    if c < max_retries:
                        filename = move_file(download_path, NEW_DOWNLOAD_FOLDER)
                    else:
                        not_downloaded.append(doc_link)
            # date_start = date_cancel = ''
            # if rows['Дата введения'] != '-' and rows['Дата введения']:
            #     date_start = datetime.strptime(rows['Дата введения'], '%d.%m.%Y').strftime('%Y-%m-%d')
            # if rows['Дата отмены'] != '-' and rows['Дата отмены']:
            #     date_cancel = datetime.strptime(rows['Дата отмены'], '%d.%m.%Y').strftime('%Y-%m-%d')
            doc_dict = {
                'number': rows['Обозначение'],
                'title': rows['Наименование'],
                'status': status_dict[rows['Статус']] if rows['Статус'] in status_dict else rows['Статус'],
                'date_start': rows['Дата введения'] if rows['Дата введения'] != '-' else '',
                'date_cancel': rows['Дата отмены'] if rows['Дата отмены'] != '-' else '',
                'replaced_by': rows['Заменен на'] if rows['Заменен на'] != '-' else '',
                'OKS': rows['Код ОКС'],
                'file_path': filename,
                'file_markdown': str(doc.find('article', {'id': 'article'})),
                'file_url': doc_link
            }
            
            global_dict[main_name][section_name].append(deepcopy(doc_dict))
            #print(doc_dict)
            #driver.quit()
            #break
        #break
    #break
driver.quit()

Global list:   0%|          | 0/40 [00:00<?, ?it/s]

Sections:   0%|          | 0/10 [00:00<?, ?it/s]

Section documents:   0%|          | 0/50 [00:00<?, ?it/s]

Section documents:   0%|          | 0/50 [00:00<?, ?it/s]

Section documents:   0%|          | 0/8 [00:00<?, ?it/s]

...

Section documents:   0%|          | 0/24 [00:00<?, ?it/s]

Section documents:   0%|          | 0/13 [00:00<?, ?it/s]

Section documents:   0%|          | 0/46 [00:00<?, ?it/s]

Section documents:   0%|          | 0/50 [00:00<?, ?it/s]

Section documents:   0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
with open('result_dict.pickle', 'wb') as file:
    pickle.dump(global_dict, file)

In [ ]:
!shutdown -s -t 30

In [10]:
for main_href, main_name in tqdm(main_list[main_start:], desc='Global list'):
    section = requests.get(url + main_href)
    time.sleep(.2)
    section = [
        (li.find('a')['href'], li.find('a').text)
        for li in bs(section.text, 'lxml').find('ul', class_='content-list').findAll('li', class_='content-list_item')
    ]
    for section_href, section_name in tqdm(section[section_start:], leave=False, desc='Sections'):
        section_docs = requests.get(url + section_href)
        time.sleep(.2)
        if section_docs.status_code == 404:
            global_dict[main_name][section_name] = 0
            continue
        lis = bs(section_docs.text, 'lxml').find('ul', class_='content-list').findAll('li', class_='content-list_item')
        if lis:
            section_docs = [(li.find('a')['href'], li.find('a').text) for li in lis]
        else:
            section_docs = []
        global_dict[main_name][section_name] = deepcopy(len(section_docs))

Global list:   0%|          | 0/40 [00:00<?, ?it/s]

Sections:   0%|          | 0/10 [00:00<?, ?it/s]

Sections:   0%|          | 0/12 [00:00<?, ?it/s]

Sections:   0%|          | 0/8 [00:00<?, ?it/s]

Sections:   0%|          | 0/11 [00:00<?, ?it/s]

...

Sections:   0%|          | 0/5 [00:00<?, ?it/s]

Sections:   0%|          | 0/14 [00:00<?, ?it/s]

Sections:   0%|          | 0/12 [00:00<?, ?it/s]

Sections:   0%|          | 0/3 [00:00<?, ?it/s]

Sections:   0%|          | 0/18 [00:00<?, ?it/s]

In [11]:
global_dict

{'01 ОБЩИЕ ПОЛОЖЕНИЯ. ТЕРМИНОЛОГИЯ. СТАНДАРТИЗАЦИЯ. ДОКУМЕНТАЦИЯ': {'01.020 Терминология (принципы и координация)': 50,
  '01.040 Словари': 1221,
  '01.060 Величины и единицы измерения': 8,
  '01.070 Цветовое кодирование': 10,
  '01.075 Знаковые обозначения': 16,
  '01.080 Графические обозначения': 194,
  '01.100 Технические чертежи': 180,
  '01.110 Техническая документация на продукцию': 199,
  '01.120 Стандартизация. Общие правила': 141,
  '01.140 Информатика. Издательское дело': 246},
 '03 УСЛУГИ. ОРГАНИЗАЦИЯ ФИРМ И УПРАВЛЕНИЕ ИМИ. АДМИНИСТРАЦИЯ. ТРАНСПОРТ. СОЦИОЛОГИЯ': {'03.020 Социология. Демография': 0,
  '03.040 Труд. Занятость': 6,
  '03.060 Финансы. Банковское дело. Денежные системы. Страхование': 14,
  '03.080 Услуги': 448,
  '03.100 Организация фирм и управление ими. Системы менеджмента': 257,
  '03.120 Качество': 939,
  '03.140 Патенты. Интеллектуальная собственность': 25,
  '03.160 Законодательство. Администрация': 7,
  '03.180 Образование': 3,
  '03.200 Досуг. Туризм': 25

In [22]:
with open('result_dict.pickle', 'rb') as file:
    result_dict = pickle.load(file)

In [1]:
with open('nums.pickle', 'wb') as file:
    pickle.dump(nums, file)

NameError: name 'pickle' is not defined

In [11]:
max_n = 10

overall_sum = 0
for _, main_section in tqdm(nums.items()):
    for sec_name, sec_num in main_section.items():
        overall_sum += min(max_n, sec_num)
print(overall_sum)

  0%|          | 0/40 [00:00<?, ?it/s]

3519


In [18]:
new_max_n = 10

In [25]:
for main_name, sections in tqdm(result_dict.items()):
    for section_name, doc_list in sections.items():
        del doc_list[new_max_n:]
    #         break
    #     break
    # break

  0%|          | 0/40 [00:00<?, ?it/s]

In [28]:
with open('res_dict_10.pickle', 'wb') as file:
    pickle.dump(result_dict, file)

In [26]:
new_nums = {}
overall_sum = 0
for main_name, sections in tqdm(result_dict.items()):
    new_nums[main_name] = {}
    for section_name, doc_list in sections.items():
        new_nums[main_name][section_name] = len(doc_list)
        overall_sum += len(doc_list)
overall_sum
    #         break
    #     break
    # break

  0%|          | 0/40 [00:00<?, ?it/s]

3519

In [29]:
old_folder = 'files'
new_folder = 'files_cropped'

In [38]:
for main_name, sections in tqdm(result_dict.items()):
    for section_name, doc_list in sections.items():
        for doc in doc_list:
            #shutil.copy(r'C:\Users\1645286\Desktop\ubuntu.txt', r'C:\Users\1645286\Desktop\ubuntu2.txt')
            if doc['file_path']:
                #print(f"{old_folder}/{doc['file_path']} -> {new_folder}/{doc['file_path']}")
                shutil.copy(f"{old_folder}/{doc['file_path']}", f"{new_folder}/{doc['file_path']}")
            #print(doc.keys())

  0%|          | 0/40 [00:00<?, ?it/s]

In [43]:
for main_name, sections in tqdm(result_dict.items()):
    for section_name, doc_list in sections.items():
        for doc in doc_list:
            if not doc['file_path']:
                print('-', doc['title'], '/', doc['file_url'])

  0%|          | 0/40 [00:00<?, ?it/s]

- Изделия медицинские. Оценка биологического действия медицинских изделий. Часть 33. Руководство по испытаниям на генотоксичность. Дополнение к ISO 10993-3 / https://allgosts.ru/01/020/gost_iso!tr_10993-33-2018
- Единая система конструкторской документации. Нанесение размеров и предельных отклонений / https://allgosts.ru/01/080/gost_2.307-2011
- Безопасность финансовых (банковских) операций. Защита информации финансовых организаций. Методика оценки соответствия / https://allgosts.ru/03/060/gost_r_57580.2-2018
- Система разработки и постановки продукции на производство. Испытания и приемка выпускаемой продукции. Основные положения / https://allgosts.ru/03/100/gost_15.309-98


In [13]:
cur_nums

{'01 ОБЩИЕ ПОЛОЖЕНИЯ. ТЕРМИНОЛОГИЯ. СТАНДАРТИЗАЦИЯ. ДОКУМЕНТАЦИЯ': {'01.020 Терминология (принципы и координация)': 50,
  '01.040 Словари': 50,
  '01.060 Величины и единицы измерения': 8,
  '01.070 Цветовое кодирование': 10,
  '01.075 Знаковые обозначения': 16,
  '01.080 Графические обозначения': 50,
  '01.100 Технические чертежи': 50,
  '01.110 Техническая документация на продукцию': 50,
  '01.120 Стандартизация. Общие правила': 50,
  '01.140 Информатика. Издательское дело': 50},
 '03 УСЛУГИ. ОРГАНИЗАЦИЯ ФИРМ И УПРАВЛЕНИЕ ИМИ. АДМИНИСТРАЦИЯ. ТРАНСПОРТ. СОЦИОЛОГИЯ': {'03.020 Социология. Демография': 0,
  '03.040 Труд. Занятость': 6,
  '03.060 Финансы. Банковское дело. Денежные системы. Страхование': 14,
  '03.080 Услуги': 50,
  '03.100 Организация фирм и управление ими. Системы менеджмента': 50,
  '03.120 Качество': 50,
  '03.140 Патенты. Интеллектуальная собственность': 25,
  '03.160 Законодательство. Администрация': 7,
  '03.180 Образование': 3,
  '03.200 Досуг. Туризм': 25,
  '03.22

In [27]:
new_nums

{'01 ОБЩИЕ ПОЛОЖЕНИЯ. ТЕРМИНОЛОГИЯ. СТАНДАРТИЗАЦИЯ. ДОКУМЕНТАЦИЯ': {'01.020 Терминология (принципы и координация)': 10,
  '01.040 Словари': 10,
  '01.060 Величины и единицы измерения': 8,
  '01.070 Цветовое кодирование': 10,
  '01.075 Знаковые обозначения': 10,
  '01.080 Графические обозначения': 10,
  '01.100 Технические чертежи': 10,
  '01.110 Техническая документация на продукцию': 10,
  '01.120 Стандартизация. Общие правила': 10,
  '01.140 Информатика. Издательское дело': 10},
 '03 УСЛУГИ. ОРГАНИЗАЦИЯ ФИРМ И УПРАВЛЕНИЕ ИМИ. АДМИНИСТРАЦИЯ. ТРАНСПОРТ. СОЦИОЛОГИЯ': {'03.020 Социология. Демография': 0,
  '03.040 Труд. Занятость': 6,
  '03.060 Финансы. Банковское дело. Денежные системы. Страхование': 10,
  '03.080 Услуги': 10,
  '03.100 Организация фирм и управление ими. Системы менеджмента': 10,
  '03.120 Качество': 10,
  '03.140 Патенты. Интеллектуальная собственность': 10,
  '03.160 Законодательство. Администрация': 7,
  '03.180 Образование': 3,
  '03.200 Досуг. Туризм': 10,
  '03.22

In [ ]:
{
    '01 ОБЩИЕ ПОЛОЖЕНИЯ. ТЕРМИНОЛОГИЯ. СТАНДАРТИЗАЦИЯ. ДОКУМЕНТАЦИЯ': {
        '01.020 Терминология (принципы и координация)': [
            {
                'number': 'ГОСТ Р 57095-2016',
                'title': 'Биотехнологии. Термины и определения',
                'status': 'ative', # replaced, cancelled, ...
                'date_start': '2017-01-05', # yyyy-mm-dd
                'date_cancel': '', # yyyy-mm-dd
                'replaced_by': '',
                'OKS': '01.020',
                'file_path': '.../downloads/gost_r_57095-2016.pdf',
                'file_markdown': '<article>...</article>',
                'file_url': 'https://allgosts.ru/01/020/gost_r_57095-2016'
            },
            {
                
            }
        ]        
    }
}